In [ ]:
####################################################
#-- Code/solutions for pencil and paper exercises --
####################################################

# Supplementary Code for: A Step-by-Step Tutorial on Active Inference Modelling and its 
# Application to Empirical Data

# By: Ryan Smith, Karl J. Friston, Christopher J. Whyte

# Note to readers: be sure to run sections individually

In [8]:
import numpy as np

In [9]:
# natural log that replaces zero values with very small values for numerical reasons.
def nat_log(x):
    x = np.where(x == 0, 1e-16, x)
    return np.log(x)

# Static perception

In [10]:
# priors
D = np.array([0.75, 0.25])

# likelihood mapping
A = np.array([[0.8, 0.2],
              [0.2, 0.8]])

# observations
o = np.array([1, 0])

# express generative model in terms of update equations
lns = nat_log(D) + nat_log(A.T @ o)

# normalize using a softmax function to find posterior
s = np.exp(lns) / np.sum(np.exp(lns))

print('Posterior over states q(s):')
print(' ')
print(s)

# Note: Because the natural log of 0 is undefined, for numerical reasons 
# the nat_log function here replaces zero values with very small values. This
# means that the answers generated by this function will vary slightly from
# the exact solutions shown in the text.

Posterior over states q(s):
 
[0.92307692 0.07692308]


# Dynamic perception

In [11]:
# priors
D = np.array([0.5, 0.5])

# likelihood mapping
A = np.array([[0.9, 0.1],
              [0.1, 0.9]])

# transitions
B = np.array([[1, 0],
              [0, 1]])

# observations  
o = {
    (1, 1): np.array([1, 0]),
    (1, 2): np.array([0, 0]),
    (2, 1): np.array([1, 0]),
    (2, 2): np.array([1, 0])
}

# number of timesteps
T = 2

# initialise posterior 
Qs = np.zeros((2, T))  # 创建一个 2xT 的零矩阵
for t in range(T):
    Qs[:, t] = np.array([0.5, 0.5])

for t in range(T):
    for tau in range(1, T + 1):
        # get correct D and B for each time point
        if tau == 1:  # first time point
            lnD = nat_log(D)  # past
            lnBs = nat_log(B.T @ Qs[:, tau])  # future
        elif tau == T:  # last time point
            lnBs = nat_log(B.T @ Qs[:, tau - 2])  # no contribution from future

        # likelihood
        lnAo = nat_log(A.T @ o[(t + 1, tau)])

        # update equation
        if tau == 1:
            lns = 0.5 * lnD + 0.5 * lnBs + lnAo
        elif tau == T:
            lns = 0.5 * lnBs + lnAo

        # normalize using a softmax function to find posterior
        Qs[:, tau - 1] = np.exp(lns) / np.sum(np.exp(lns))

print('Final posterior beliefs over states:')
print(Qs)

Final posterior beliefs over states:
[[0.93971712 0.97262824]
 [0.06028288 0.02737176]]
